In [38]:
%cd data

/home/tetsuya/repo/programming-study/aws_sagemaker/sagemaker_jparacrawl_mecab/data


In [3]:
!wget https://www.kecl.ntt.co.jp/icl/lirg/jparacrawl/release/3.0/bitext/en-ja.tar.gz

--2022-11-25 18:41:54--  https://www.kecl.ntt.co.jp/icl/lirg/jparacrawl/release/3.0/bitext/en-ja.tar.gz
www.kecl.ntt.co.jp (www.kecl.ntt.co.jp) をDNSに問いあわせています... 163.137.218.58
www.kecl.ntt.co.jp (www.kecl.ntt.co.jp)|163.137.218.58|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 2972770796 (2.8G) [application/x-gzip]
`en-ja.tar.gz' に保存中

en-ja.tar.gz        100%[===================>]   2.77G  4.06MB/s    in 10m 52s 

2022-11-25 18:52:46 (4.35 MB/s) - `en-ja.tar.gz' へ保存完了 [2972770796/2972770796]



In [4]:
!tar xvf en-ja.tar.gz

In [39]:
%cd en-ja/

/home/tetsuya/repo/programming-study/aws_sagemaker/sagemaker_jparacrawl_mecab/data/en-ja


In [6]:
!cut -f 1 en-ja.bicleaner05.txt | uniq | wc -l

64394


In [7]:
!cut -f 1 en-ja.bicleaner05.txt | uniq | head

0001vip.cocolog-nifty.com
000-lhr.web.wox.cc
0017.org
001sprint.com
0024.review.wox.cc
011300.web.wox.cc
01e-coach.roundtable.jp
0257.jp
0327.r-fs11136.net
03-willcom.windows-keitai.com
uniq: 書き込みエラー: Broken pipe


In [8]:
!cat en-ja.bicleaner05.txt | cut -f4 > en.txt
!cat en-ja.bicleaner05.txt | cut -f5 > ja.txt

In [9]:
!pip install mecab-python3
!pip install unidic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.1/581.1 kB 9.2 MB/s eta 0:00:009.0 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7409 sha256=519dada52465ae848c867a2191c2110c2c0d468124b0d89264470844b258a3e0
  Stored in directory: /home/tetsuya/.cache/pip/wheels/7a/72/72/1f3d654c345ea69d5d51b531c90daf7ba14cc555eaf2c64ab0
Successfully built unidic


In [10]:
!python -m unidic download

download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100%|██████████████████████| 526M/526M [01:00<00:00, 8.73MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /home/tetsuya/anaconda3/envs/sagemaker/lib/python3.10/site-packages/unidic/dicdir


In [11]:
import MeCab
import unidic

tagger = MeCab.Tagger()  # 「tagger = MeCab.Tagger('-d ' + unidic.DICDIR)」
sample_txt = 'くるまでまつ'
result = tagger.parse(sample_txt)
print(result)

くる	動詞,非自立可能,,,カ行変格,連体形-一般,クル,来る,くる,クル,くる,クル,和,"","","","","","",用,クル,クル,クル,クル,"1","C1","",2891174448865985,10518
まで	助詞,副助詞,,,,,マデ,まで,まで,マデ,まで,マデ,和,"","","","","","",副助,マデ,マデ,マデ,マデ,"","名詞%F2@1,形容詞%F2@1,動詞%F2@1","",9865651581755904,35891
まつ	動詞,一般,,,五段-タ行,連体形-一般,マツ,待つ,まつ,マツ,まつ,マツ,和,"","","","","","",用,マツ,マツ,マツ,マツ,"1","C1","",9848884029432513,35830
EOS



In [17]:
wakati = MeCab.Tagger('-Owakati')
result = wakati.parse(sample_txt)
print(sample_txt)
print(result)

くるまでまつ
くる まで まつ 



In [33]:
with open("./ja.txt") as in_f:
    with open("ja_tokenized.txt", mode='w') as out_f:
        for line in in_f:
            out_f.write(wakati.parse(line))

In [12]:
!pip install sentencepiece

In [13]:
!shuf -n 100000 en.txt -o vocab_train.en

In [14]:
import sentencepiece as spm

spm.SentencePieceTrainer.Train(
   '--input=vocab_train.en --model_prefix=sentencepiece_en --vocab_size=32000 --character_coverage=0.98'
)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=vocab_train.en --model_prefix=sentencepiece_en --vocab_size=32000 --character_coverage=0.98
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: vocab_train.en
  input_format: 
  model_prefix: sentencepiece_en
  model_type: UNIGRAM
  vocab_size: 32000
  self_test_sample_size: 0
  character_coverage: 0.98
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_

In [ ]:
sp_en = spm.SentencePieceProcessor("sentencepiece_en.model")

with open("./en.txt") as in_f:
    with open("en_tokenized.txt", mode='w') as out_f:
        for line in in_f:
            out_f.write(' '.join(sp_en.encode(line, out_type=str)) + "\n")

In [19]:
!head en_tokenized.txt

▁And ▁everyone ▁will ▁not ▁care ▁that ▁it ▁is ▁not ▁you .
▁And ▁everyone ▁will ▁not ▁care ▁that ▁it ▁is ▁not ▁you .
▁Sponsor ed ▁link ▁This ▁advertisement ▁is ▁displayed ▁when ▁there ▁is ▁no ▁update ▁for ▁a ▁certain ▁period ▁of ▁time .
▁Also , ▁it ▁will ▁always ▁be ▁hidden ▁when ▁becoming ▁a ▁premium ▁user ▁ .
▁It ▁will ▁return ▁to ▁non - display ▁when ▁content ▁update ▁is ▁done .
▁Sponsor ed ▁link ▁This ▁advertisement ▁is ▁displayed ▁when ▁there ▁is ▁no ▁update ▁for ▁a ▁certain ▁period ▁of ▁time .
▁Also , ▁it ▁will ▁always ▁be ▁hidden ▁when ▁becoming ▁a ▁premium ▁user ▁ .
▁It ▁will ▁return ▁to ▁non - display ▁when ▁content ▁update ▁is ▁done .
▁It ’ s ▁like ▁you ▁can ▁enrich ▁it ▁and ▁save ▁money ▁as ▁a ▁result . ▁I ’ ve ▁watched ▁a ▁lot ▁of ▁videos , ▁mainly ▁on ▁ Y outube , ▁of ▁people ▁who ▁say ▁they ’ re ▁minimalist s , ▁and ▁I ▁agree .
▁ G o ▁to ▁the ▁original ▁video ▁hierarchy ▁of ▁the ▁conversion ▁source , ▁copy ▁and ▁paste ▁the ▁following ▁is ▁fine . ▁ ff m peg ▁ - i ▁sample . 

In [34]:
!head ja_tokenized.txt

鼻 ・ 口 の ところ は あらかじめ 少し 切っ て おく と いい です ね 。 
アドレス 置い とく の で 、 消さ れ ない うち に メール くれ たら 嬉しい です 。 
スポンサー ドリンク この 広告 は 一定 期間 更新 が ない 場合 に 表示 さ れ ます 。 
また 、 プレミアム ユーザー に なる と 常 に 非 表示 に なり ます 。 
コンテンツ の 更新 が 行わ れる と 非 表示 に 戻り ます 。 
スポンサー ドリンク この 広告 は 一定 期間 更新 が ない 場合 に 表示 さ れ ます 。 
また 、 プレミアム ユーザー に なる と 常 に 非 表示 に なり ます 。 
コンテンツ の 更新 が 行わ れる と 非 表示 に 戻り ます 。 
You tube を 中心 に ミニマリスト と 言っ て いる 方 の 動画 を たくさん み まし た が 、 納得 いく もの の も 多く 、 不況 と 少子 化 を 呼ば れる ” 今 の ” 日本 に は 合っ て いる 考え 方 な の か な 、 と 感じ て い ます 。 
ffmpeg - i sample . mp 4 - strict - 2 video . webm まとめ Web 上 で 動画 を 設置 する とき は 、 You tube に アップ し て 埋め込む 方法 ばかり 使っ て い まし た が 、 これ で 複数 の 動画 形式 を 作る こと が できる の で 、 自分 の サーバ に 設定 する こと も 可能 に なり ます ね 。 


In [21]:
!seq `wc -l en-ja.bicleaner05.txt | cut -f1 -d' '` | shuf -o line_nums.txt -

In [22]:
!head --lines=25730000 line_nums.txt | sort --numeric-sort > train_line_nums.txt
!head --lines=25735000 line_nums.txt | tail --lines=5000 | sort --numeric-sort > val_line_nums.txt
!head --lines=25740000 line_nums.txt | tail --lines=5000 | sort --numeric-sort > test_line_nums.txt

In [23]:
# input_file: 取り出し元ファイル
# num_fileの行番号の行だけ取り出します。
# output_file: 取り出した結果ファイル
def extract_lines(input_file, num_file, output_file):
    text_line_num = 1
    with open(num_file) as line_f:
        line_num = int(line_f.readline())
        with open(input_file) as in_f:
            with open(output_file, mode='w') as out_f:
                for line in in_f:
                    if text_line_num == line_num:
                        out_f.write(line)
                        line_num = line_f.readline()
                        if line_num == '':
                            break
                        else:
                            line_num = int(line_num)
                    text_line_num += 1

In [24]:
extract_lines("en_tokenized.txt", "train_line_nums.txt", "train.en")
extract_lines("en_tokenized.txt", "val_line_nums.txt", "val.en")
extract_lines("en_tokenized.txt", "test_line_nums.txt", "test.en")

In [40]:
extract_lines("ja_tokenized.txt", "train_line_nums.txt", "train.ja")
extract_lines("ja_tokenized.txt", "val_line_nums.txt", "val.ja")
extract_lines("ja_tokenized.txt", "test_line_nums.txt", "test.ja")

In [42]:
%cd ../..

/home/tetsuya/repo/programming-study/aws_sagemaker/sagemaker_jparacrawl_mecab


In [44]:
!onmt_build_vocab -config vocab_en_ja.yml -n_sample 10000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-11-25 21:37:56,413 INFO] Counter vocab from 10000 samples.
[2022-11-25 21:37:56,413 INFO] Build vocab on 10000 transformed examples/corpus.
[2022-11-25 21:37:56,419 INFO] corpus_1's transforms: TransformPipe()
[2022-11-25 21:37:56,694 INFO] Counters src:10532
[2022-11-25 21:37:56,694 INFO] Counters tgt:11431


In [49]:
!mkdir -p data_train/en-ja
!cp data/en-ja/jparacrawl.vocab.* data_train/en-ja
!cp data/en-ja/train.* data_train/en-ja
!cp data/en-ja/val.* data_train/en-ja

In [14]:
!./build_and_push.sh jparacrawl-train

WARNING! Your password will be stored unencrypted in /home/tetsuya/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  1.171MB
Step 1/9 : FROM nvidia/cuda:11.7.1-runtime-ubuntu20.04
 ---> 9a178fee7c22
Step 2/9 : RUN apt-get -y update
 ---> Using cache
 ---> 4d7452ab4cd0
Step 3/9 : RUN apt-get -y install python3 pip
 ---> Using cache
 ---> 8ed2c75c4f94
Step 4/9 : RUN pip --no-cache-dir install OpenNMT-py
 ---> Using cache
 ---> ccfcabb82d71
Step 5/9 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> b39c6aad63a5
Step 6/9 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> c8cb8c7e5898
Step 7/9 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> d43f14365ed3
Step 8/9 : COPY src /opt/program
 ---> f1cb1c16fcd6
Step 9/9 : WORKDIR /opt/program
 ---> Running in ff9abe4d0705
Removing intermediate container ff9abe4d0705


In [2]:
import boto3

role_name = "SageMaker-local"

iam = boto3.client("iam")
role = iam.get_role(RoleName=role_name)["Role"]["Arn"]

In [1]:
import sagemaker as sage

sess = sage.Session()

In [4]:
prefix = 'jparacrawl/training'
WORK_DIRECTORY = 'data_train'
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [15]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/jparacrawl-train:latest'.format(account, region)

estimator = sage.estimator.Estimator(image,
                        role, 1, 'ml.g4dn.12xlarge',
                        output_path="s3://{}/jparacrawl/output".format(sess.default_bucket()),
                        sagemaker_session=sess)

estimator.fit({"training": data_location})

2022-11-26 02:29:38 Starting - Starting the training job...
2022-11-26 02:30:06 Starting - Preparing the instances for trainingProfilerReport-1669429777: InProgress
......
2022-11-26 02:31:08 Downloading - Downloading input data..............[2022-11-26 02:33:37,992 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2022-11-26 02:33:37,992 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-11-26 02:33:37,992 INFO] Missing transforms field for valid data, set to default: [].
[2022-11-26 02:33:37,992 INFO] Parsed 2 corpora from -data.
[2022-11-26 02:33:37,992 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-11-26 02:33:37,992 INFO] Loading vocab from text file...
[2022-11-26 02:33:37,992 INFO] Loading src vocabulary from /opt/ml/input/data/training/en-ja/jparacrawl.vocab.src
[2022-11-26 02:33:38,009 INFO] Loaded src vocab has 10532 tokens.
[2022-11-26 02:33:38,013 INFO] Loading tgt vocabulary from /opt/ml/

In [2]:
!mkdir data_model

In [6]:
prefix = 'jparacrawl/output/jparacrawl-train-2022-11-26-02-29-37-291/output/model.tar.gz'
MODEL_DIRECTORY = 'data_model'
sess.download_data(MODEL_DIRECTORY, sess.default_bucket(), key_prefix=prefix)

# sess.download_data(MODEL_DIRECTORY, sess.default_bucket(), key_prefix=estimator.model_data)

In [7]:
%cd data_model

/home/tetsuya/repo/programming-study/aws_sagemaker/sagemaker_jparacrawl_mecab/data_model


In [8]:
!tar xvf model.tar.gz

model.en-ja_step_40000.pt
model.en-ja_step_10000.pt
model.en-ja_step_20000.pt
model.en-ja_step_30000.pt


In [9]:
%cd ..

/home/tetsuya/repo/programming-study/aws_sagemaker/sagemaker_jparacrawl_mecab


In [11]:
!cp data/en-ja/sentencepiece_en.model data_serve/

In [12]:
%cd local_translate

/home/tetsuya/repo/programming-study/aws_sagemaker/sagemaker_jparacrawl_mecab/local_translate


In [17]:
!chmod +x src/translate.py

In [19]:
!docker build -t ctarnslate2_jpara_mecab:latest .

Sending build context to Docker daemon  6.656kB
Step 1/8 : FROM ubuntu:22.04
 ---> 2dc39ba059dc
Step 2/8 : RUN apt-get -y update
 ---> Using cache
 ---> c7133dee31b8
Step 3/8 : RUN apt-get -y install python3 pip
 ---> Using cache
 ---> ec402a346ff5
Step 4/8 : RUN pip install ctranslate2 OpenNMT-py sentencepiece
 ---> Using cache
 ---> 0dacd89ab0b6
Step 5/8 : COPY src/translate.py /usr/bin/
 ---> 87cbfdbae61e
Step 6/8 : WORKDIR /var/opt/ctranslate2
 ---> Running in 51917df973d1
Removing intermediate container 51917df973d1
 ---> b0c1fda78bd1
Step 7/8 : ENTRYPOINT [ "translate.py" ]
 ---> Running in 8913963c315c
Removing intermediate container 8913963c315c
 ---> e479d7710ca7
Step 8/8 : CMD [ "Hello world!" ]
 ---> Running in 5b59a8aef6dc
Removing intermediate container 5b59a8aef6dc
 ---> bcff91a468f8
Successfully built bcff91a468f8
Successfully tagged ctarnslate2_jpara_mecab:latest


In [22]:
!cd ../data_serve

In [23]:
!docker run --rm -v $PWD:/var/opt/ctranslate2 ctarnslate2_jpara_mecab:latest 'Hello world'

Traceback (most recent call last):
  File "/usr/bin/translate.py", line 7, in <module>
    translator = ctranslate2.Translator("ctranslate2_model", device="cpu")
RuntimeError: Unable to open file 'model.bin' in model 'ctranslate2_model'
